In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from train_full_rl import load_ext_net

In [10]:
extractor, agent_vocab = load_ext_net("./cnn/extm_cnn/")

loading checkpoint ckpt-2.486399-18000...


In [11]:
from model.rl import ActorCritic
from decoding import Abstractor, ArticleBatcher
import torch
cuda = torch.cuda.is_available()
agent = ActorCritic(extractor._sent_enc,
                        extractor._art_enc,
                        extractor._extractor,
                        ArticleBatcher(agent_vocab, cuda))
if cuda:
    agent = agent.cuda()

In [12]:
# import json
# source1 = json.load(open("/exp/yashgupta/cnndm/finished_files/train/20.json", "r"))["article"] 
# tgt1 = json.load(open("/exp/yashgupta/cnndm/finished_files/train/20.json", "r"))["abstract"] 
# print(len(source1))
# print(len(tgt1))

In [13]:
# source1.to(cuda)
# agent(source1)

In [14]:
cuda

True

In [15]:
from data.data import CnnDmDataset
DATA_DIR = "/exp/yashgupta/cnndm/finished_files/"
class RLDataset(CnnDmDataset):
    """ get the article sentences only (for decoding use)"""
    def __init__(self, split):
        super().__init__(split, DATA_DIR)

    def __getitem__(self, i):
        js_data = super().__getitem__(i)
        art_sents = js_data['article']
        abs_sents = js_data['abstract']
        return art_sents, abs_sents

In [16]:
from torch.utils.data import DataLoader
# loader = DataLoader(
#         RLDataset('train'), batch_size=1,
#         shuffle=True, num_workers=4,
#         collate_fn=coll
#     )

In [17]:
# from train_full_rl import build_batchers
from itertools import cycle
from toolz.sandbox.core import unzip
from data.batcher import tokenize
def build_batchers(batch_size):
    def coll(batch):
        art_batch, abs_batch = unzip(batch)
        art_sents = list(filter(bool, map(tokenize(None), art_batch)))
        abs_sents = list(filter(bool, map(tokenize(None), abs_batch)))
        return art_sents, abs_sents
    loader = DataLoader(
        RLDataset('train'), batch_size=batch_size,
        shuffle=True, num_workers=4,
        collate_fn=coll
    )
    val_loader = DataLoader(
        RLDataset('val'), batch_size=batch_size,
        shuffle=False, num_workers=4,
        collate_fn=coll
    )
    return cycle(loader), val_loader
# DATA_DIR = "/exp/yashgupta/cnndm/finished_files/"
train_batcher, val_batcher = build_batchers(1)

In [18]:
art_batch, abs_batch = next(train_batcher)
for raw_arts in art_batch:
    # print(raw_arts)
    (inds, ms), bs = agent(raw_arts)

In [24]:
bs

[tensor([[-0.0758]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2107]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2464]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2656]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2692]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2716]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2721]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2720]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2715]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2708]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2698]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2689]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2682]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2677]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2674]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[-0.2672]], device='cuda:0', gr

In [19]:
len(inds), len(ms), len(bs)

(19, 19, 19)

In [18]:
ms[0]
# len(abs_batch[0])

Categorical(probs: torch.Size([1, 17]))

In [19]:
bs

[tensor([[0.3297]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.1225]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.1051]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0866]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0585]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0525]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0497]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0487]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0480]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0475]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0471]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0467]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0462]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0458]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0454]], device='cuda:0', grad_fn=<AddmmBackward>),
 tensor([[0.0452]], device='cuda:0', grad_fn=<AddmmBack

In [20]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../PreSumm/src/')))

In [21]:
import myextract, importlib
importlib.reload(myextract)

gpu_rank 0
gpu_rank 0


<module 'myextract' from '/exp/yashgupta/PreSumm/src/myextract.py'>

In [4]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../PreSumm/src/')))
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [5]:
import argparse
import json
import os
from os.path import join
from datetime import timedelta
from time import time

from cytoolz import identity

import torch
from torch.utils.data import DataLoader

from data.batcher import tokenize

from decoding import Abstractor, Extractor#, DecodeDataset
from decoding import make_html_safe
MAX_ABS_NUM = 6  # need to set max sentences to extract for non-RL extractor

In [13]:
from data.data import CnnDmDataset
DATASET_DIR = "/exp/yashgupta/cnndm2/finished_files/" #CHANGE BACK
class DecodeDataset(CnnDmDataset):
    """ get the article sentences only (for decoding use)"""
    def __init__(self, split):
        assert split in ['val', 'test']
        super().__init__(split, DATASET_DIR)

    def __getitem__(self, i):
        js_data = super().__getitem__(i)
        art_sents = js_data['article']
        abs_sents = js_data['abstract']
        return art_sents, abs_sents

In [14]:
abstractor = Abstractor("./cnn2/absm_cnn/", 50, True)

loading checkpoint ckpt-3.053677-87000...


In [15]:
split = "test"
import os
os.environ['DATA'] = "/exp/yashgupta/cnndm2/finished_files/"
DATASET_DIR = os.environ["DATA"]
def coll(batch):
        articles = list(filter(bool, batch))
        return articles
dataset = DecodeDataset(split)

In [16]:
n_data = len(dataset)
loader = DataLoader(
    dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=coll
)

In [17]:
n_data

11490

In [11]:
import myextract, importlib
importlib.reload(myextract)


gpu_rank 0
gpu_rank 0


<module 'myextract' from '/exp/yashgupta/PreSumm/src/myextract.py'>

In [19]:
# from tdqm.notebook import tqdm
i=0
batch_size = 1
save_path = "/exp/yashgupta/fast_abs_rl/cnn2/dec_cnn_presumm_abs"
start = time()
with torch.no_grad():
    for i_debug, x in enumerate(loader):
        # print(x)
        [(raw_article_batch, raw_abs_batch)] = x
        # print(raw_abs_batch)
        # tokenized_article_batch, tokenized_abs_batch = map(tokenize(None), [raw_article_batch]), map(tokenize(None),  [raw_abs_batch])
        # print(tokenized_abs_batch)
        ext_arts = []
        ext_inds = []
        for raw_art_sents, raw_abs_sents in zip([raw_article_batch], [raw_abs_batch]):
            # print(raw_art_sents, raw_abs_sents)
            ext = myextract.extractor([raw_art_sents], [raw_abs_sents])
            ext_inds += [(len(ext_arts), len(ext))]
            ext_arts += ext #list(map(lambda i: raw_art_sents[i], ext))
        dec_outs = abstractor(ext_arts)
        assert i == batch_size*i_debug
        # print(i)
        for j, n in ext_inds:
            decoded_sents = [' '.join(dec) for dec in dec_outs[j:j+n]]
            for k, dec_str in enumerate(decoded_sents):
                with open(join(save_path, 'output/{}.dec'.format(i)),
                            'w') as f:
                    f.write(make_html_safe(dec_str))

            i += 1
            if (i%100==0):
                print(i)
            print('{}/{} ({:.2f}%) decoded in {} seconds\r'.format(
                i, n_data, i/n_data*100, timedelta(seconds=int(time()-start))
            ), end='')
    print()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


KeyboardInterrupt: 